# ml zoomcamp
## week 8 homework
## Topic: Neural Networks and Deep Learning
### name: Isaac Muturi
### email: ndirangumuturi749@gmail.com

### Dataset

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. 
For this, we will use the "Bee or Wasp?" dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/jerzydziewierz/bee-vs-wasp) and slightly rebuilt. 

You can download the dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
unzip data.zip
```

In [1]:
# !wget -q https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# !unzip data.zip
# !rm data.zip
# !ls

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

> **Note:** you will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). 
> You can also use a computer without a GPU (e.g. your laptop), but it will be slower.


### Data Preparation

The dataset contains around 2500 images of bees and around 2100 images of wasps. 

The dataset contains separate folders for training and test sets. 


### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD

# Create a Sequential model
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))

# Add max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Add a dense layer with 64 neurons and 'relu' activation
model.add(Dense(units=64, activation='relu'))

# Output layer with 1 neuron and appropriate activation for binary classification (e.g., 'sigmoid')
model.add(Dense(units=1, activation='sigmoid'))

# Create an optimizer with SGD
optimizer = SGD(learning_rate=0.002, momentum=0.8)

# Compile the model with SGD optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


2023-11-16 20:48:59.123662: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 20:49:00.667436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

2023-11-16 20:49:00.673670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-16 20:49:00.673879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-16 20:49:00.674332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 20:49:00.674655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy`
* `categorical crossentropy`
* `cosine similarity`

> **Note:** since we specify an activation for the output layer, we don't need to set `from_logits=True`

### Question 2

What's the number of parameters in the convolutional layer of our model? You can use the `summary` method for that. 

* 1 
* 65
* 896
* 11214912

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [4]:
from keras.preprocessing.image import ImageDataGenerator

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Specify the directory for training and test data
train_generator = train_datagen.flow_from_directory('data/train',
                                                    target_size=(150, 150),
                                                    batch_size=20, 
                                                    class_mode='binary'
                                                   )
test_generator = test_datagen.flow_from_directory('data/test', 
                                                  target_size=(150, 150), 
                                                  batch_size=20, 
                                                  class_mode='binary'
                                                 )

#train the model
history = model.fit(train_generator,
          epochs=10,
          validation_data=test_generator
         )


Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.
Epoch 1/10


2023-11-16 20:49:02.703254: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-11-16 20:49:03.293914: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-16 20:49:03.294368: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-16 20:49:03.294405: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-11-16 20:49:03.294852: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-16 20:49:03.294942: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


184/184 [==============================] - 10s 42ms/step - loss: 0.6723 - accuracy: 0.5646 - val_loss: 0.6225 - val_accuracy: 0.6645
Epoch 2/10
184/184 [==============================] - 7s 41ms/step - loss: 0.6291 - accuracy: 0.6494 - val_loss: 0.5981 - val_accuracy: 0.6721
Epoch 3/10
184/184 [==============================] - 7s 40ms/step - loss: 0.5809 - accuracy: 0.6957 - val_loss: 0.5597 - val_accuracy: 0.7026
Epoch 4/10
184/184 [==============================] - 7s 40ms/step - loss: 0.5499 - accuracy: 0.7240 - val_loss: 0.5527 - val_accuracy: 0.7440
Epoch 5/10
184/184 [==============================] - 7s 40ms/step - loss: 0.5092 - accuracy: 0.7615 - val_loss: 0.5375 - val_accuracy: 0.7375
Epoch 6/10
184/184 [==============================] - 7s 40ms/step - loss: 0.4911 - accuracy: 0.7811 - val_loss: 0.5454 - val_accuracy: 0.7505
Epoch 7/10
184/184 [==============================] - 8s 41ms/step - loss: 0.4854 - accuracy: 0.7778 - val_loss: 0.5431 - val_accuracy: 0.7429
Epoch 8/1

### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80


In [5]:
import numpy as np

np.median(history.history['accuracy'])

0.7696491777896881

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.031
* 0.061
* 0.091
* 0.131


In [6]:
np.std(history.history['loss'])

0.08098069949361864

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [7]:
# ImageDataGenerator with augmentations
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator_augmented = train_datagen_augmented.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'  # Use 'binary' for binary classification
)

# use augmented generator for training
history = model.fit(
    train_generator_augmented,
    epochs=10,
    validation_data=test_generator  # 'test_generator' doesnt change
)


Found 3677 images belonging to 2 classes.
Epoch 1/10
184/184 [==============================] - 21s 114ms/step - loss: 0.4966 - accuracy: 0.7696 - val_loss: 0.4766 - val_accuracy: 0.7756
Epoch 2/10
184/184 [==============================] - 21s 113ms/step - loss: 0.4865 - accuracy: 0.7732 - val_loss: 0.4882 - val_accuracy: 0.7680
Epoch 3/10
184/184 [==============================] - 21s 114ms/step - loss: 0.4824 - accuracy: 0.7822 - val_loss: 0.4594 - val_accuracy: 0.7887
Epoch 4/10
184/184 [==============================] - 21s 114ms/step - loss: 0.4757 - accuracy: 0.7849 - val_loss: 0.4880 - val_accuracy: 0.7680
Epoch 5/10
184/184 [==============================] - 21s 114ms/step - loss: 0.4729 - accuracy: 0.7832 - val_loss: 0.4734 - val_accuracy: 0.7821
Epoch 6/10
184/184 [==============================] - 21s 114ms/step - loss: 0.4752 - accuracy: 0.7775 - val_loss: 0.4846 - val_accuracy: 0.7582
Epoch 7/10
184/184 [==============================] - 21s 113ms/step - loss: 0.4642 - ac

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108

In [8]:
np.mean(history.history['val_loss'])

0.4709283709526062

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.38
* 0.58
* 0.78
* 0.98

In [9]:
np.mean(history.history['val_accuracy'][5:])

0.7806100249290466

In [10]:
!nvidia-smi

Thu Nov 16 20:58:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    31W /  70W |  14631MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------